In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import matplotlib.pyplot as plt
import seaborn as sns

Downloads

In [16]:
pip install thundersvm

  Obtaining dependency information for thundersvm from https://files.pythonhosted.org/packages/7d/16/281a54f6d1f70c59df242f2f93e5cc04daf01b9c9809c2b154d15ea6a346/thundersvm-0.3.12-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/507.4 kB ? eta -:--:--
    --------------------------------------- 10.2/507.4 kB ? eta -:--:--
   - ------------------------------------- 20.5/507.4 kB 217.9 kB/s eta 0:00:03
   --- ----------------------------------- 51.2/507.4 kB 440.4 kB/s eta 0:00:02
   ------------ --------------------------- 153.6/507.4 kB 1.0 MB/s eta 0:00:01
   ------------------- -------------------- 245.8/507.4 kB 1.3 MB/s eta 0:00:01
   ----------------------------- ---------- 368.6/507.4 kB 1.4 MB/s eta 0:00:01
   ------------------------------------- -- 481.3/507.4 kB 1.6 MB/s eta 0:00:01
   ---------------------------------------- 507.4/507.4 kB 1.4 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip uninstall thundersvm

In [ ]:
git clone https://github.com/Xtra-Computing/thundersvm.git
cd thundersvm/python
python setup.py install

## Pre-processing

In [2]:
dtype = {
    'ResponseID': 'category',
    'UserID': 'int8',
    'Intervention': 'int8',
    'PedPed': 'int8',
    'Barrier': 'int8',
    'CrossingSignal': 'int8',
    'AttributeLevel': 'category',
    'ScenarioTypeStrict': 'category',
    'NumberOfCharacters': 'int8',
    'DiffNumberOFCharacters': 'int8',
    'Saved': 'int8',
    'Man': 'int8',
    'Woman': 'int8',
    'Pregnant': 'int8',
    'Stroller': 'int8',
    'OldMan': 'int8',
    'OldWoman': 'int8',
    'Boy': 'int8',
    'Girl': 'int8',
    'Homeless': 'int8',
    'LargeWoman': 'int8',
    'LargeMan': 'int8',
    'Criminal': 'int8',
    'MaleExecutive': 'int8',
    'FemaleExecutive': 'int8',
    'FemaleAthlete': 'int8',
    'MaleAthlete': 'int8',
    'FemaleDoctor': 'int8',
    'MaleDoctor': 'int8',
    'Dog': 'int8',
    'Cat': 'int8'
}


In [3]:
df50 = pd.read_csv('total_50_dataset.csv', dtype=dtype)

In [4]:
df50.shape

(2500000, 31)

In [5]:
# preprocessing

# one-hot encode the AttributeLevel and ScenarioTypeStrict
df50 = pd.get_dummies(df50, columns=['AttributeLevel', 'ScenarioTypeStrict'])

print(df50.columns)


Index(['ResponseID', 'UserID', 'Intervention', 'PedPed', 'Barrier',
       'CrossingSignal', 'NumberOfCharacters', 'DiffNumberOFCharacters',
       'Saved', 'Man', 'Woman', 'Pregnant', 'Stroller', 'OldMan', 'OldWoman',
       'Boy', 'Girl', 'Homeless', 'LargeWoman', 'LargeMan', 'Criminal',
       'MaleExecutive', 'FemaleExecutive', 'FemaleAthlete', 'MaleAthlete',
       'FemaleDoctor', 'MaleDoctor', 'Dog', 'Cat', 'AttributeLevel_Fat',
       'AttributeLevel_Female', 'AttributeLevel_Fit', 'AttributeLevel_High',
       'AttributeLevel_Hoomans', 'AttributeLevel_Less', 'AttributeLevel_Low',
       'AttributeLevel_Male', 'AttributeLevel_More', 'AttributeLevel_Old',
       'AttributeLevel_Pets', 'AttributeLevel_Young', 'ScenarioTypeStrict_Age',
       'ScenarioTypeStrict_Fitness', 'ScenarioTypeStrict_Gender',
       'ScenarioTypeStrict_Social Status', 'ScenarioTypeStrict_Species',
       'ScenarioTypeStrict_Utilitarian'],
      dtype='object')


In [6]:
df50.shape

(2500000, 47)

In [7]:
df50.duplicated().sum()

0

In [14]:
# now also dropping ResponseID
# kept this on and pre-processed it thus far to keep track of if everything went right with the complete sessions being in the dataset (so each ResponseID has to be present twice)

#### Splitting

In [8]:
# Prepare features and target variable
X = df50.drop(['UserID', 'ResponseID'], axis=1)     # Features
y = df50['UserID']                                  # Target variable

In [9]:
# Split the data into train, validation and test sets
X_trainval, X_test, y_trainval, y_test = train_test_split(X, y, test_size=0.15, random_state=42)

In [10]:
X_trainval.shape

(2125000, 45)

In [11]:
# Setting K for K-fold cross validation

from sklearn.model_selection import KFold

k = 3
kf = KFold(n_splits=k, random_state=42, shuffle=True)

## Modelling

### Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
# Initialize logistic regression model
lr_model = LogisticRegression(max_iter=1000)

# Lists to store results
conf_matrices_lr = []
classification_reports_lr = []
count = 0

# Loop through each fold
for train_index, test_index in kf.split(X_trainval):
    count += 1

    # Split data into train and test for this fold
    X_train, X_val = X_trainval.iloc[train_index], X_trainval.iloc[test_index]
    y_train, y_val = y_trainval.iloc[train_index], y_trainval.iloc[test_index]

    # Train the model
    lr_model.fit(X_train, y_train)

    # Make predictions
    y_pred_lr = lr_model.predict(X_val)

    # Generate confusion matrix and classification report
    conf_matrices_lr.append(confusion_matrix(y_val, y_pred_lr))
    classification_reports_lr.append(classification_report(y_val, y_pred_lr, output_dict=True))

    print("Done processing fold " + str(count))

Done processing fold 1
Done processing fold 2
Done processing fold 3


In [ ]:
# Display results
for i in range(k):
    print(f"\nFold {i+1}")
    print("Confusion Matrix:")
    print(conf_matrices_lr[i])
    print("\nClassification Report:")
    for label, metrics in classification_reports_lr[i].items():
        print(f"{label}: {metrics}")


Fold 1
Confusion Matrix:
[[264268  89554]
 [195688 158824]]

Classification Report:
0: {'precision': 0.574550609188705, 'recall': 0.7468953315508928, 'f1-score': 0.6494842573773191, 'support': 353822.0}
1: {'precision': 0.6394447173260112, 'recall': 0.4480074017240601, 'f1-score': 0.5268755494368791, 'support': 354512.0}
accuracy: 0.5973057907710204
macro avg: {'precision': 0.6069976632573582, 'recall': 0.5974513666374764, 'f1-score': 0.5881799034070991, 'support': 708334.0}
weighted avg: {'precision': 0.6070292704755734, 'recall': 0.5973057907710204, 'f1-score': 0.5881201858102599, 'support': 708334.0}

Fold 2
Confusion Matrix:
[[265489  88962]
 [194741 159141]]

Classification Report:
0: {'precision': 0.5768615692153923, 'recall': 0.7490146733963228, 'f1-score': 0.6517618552537742, 'support': 354451.0}
1: {'precision': 0.6414311797922637, 'recall': 0.44970074770686275, 'f1-score': 0.5287208153027069, 'support': 353882.0}
accuracy: 0.5994779291660843
macro avg: {'precision': 0.609146

In [ ]:
# Calculate average accuracy
accuracies_lr = [report['accuracy'] for report in classification_reports_lr]
average_accuracy_lr = np.mean(accuracies_lr)
print(f"\nAverage Accuracy LR: {average_accuracy_lr}")

# Caluculate average standard deviation
std_dev_lr = np.std(accuracies_lr)
print(f"Standard Deviation: {std_dev_lr}")


Average Accuracy LR: 0.5986701182890953
Standard Deviation: 0.0009701647216432922


In [ ]:
# Collect the recall for the '1' class from each fold
recall_class_1_lr = [report['1']['recall'] for report in classification_reports_lr]

# Calculate the average recall for the '1' class across all folds
average_recall_class_1_lr = np.mean(recall_class_1_lr)
print(f"\nAverage Recall for class 1: {average_recall_class_1_lr}")


Average Recall for class 1: 0.44855839320270907


### Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
# Initialize Random Forest model
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)

# Lists to store results
conf_matrices_rf = []
classification_reports_rf = []
count = 0

# Manually loop through each fold
for train_index, test_index in kf.split(X_trainval):
    count += 1
    # Split the data for this fold
    X_train, X_val = X_trainval.iloc[train_index], X_trainval.iloc[test_index]
    y_train, y_val = y_trainval.iloc[train_index], y_trainval.iloc[test_index]

    # Train the model
    rf_model.fit(X_train, y_train)

    # Make predictions
    y_pred_rf = rf_model.predict(X_val)

    # Store confusion matrix and classification report for each fold
    conf_matrices_rf.append(confusion_matrix(y_val, y_pred_rf))
    classification_reports_rf.append(classification_report(y_val, y_pred_rf, output_dict=True))

    print("Done processing fold" + str(count))

Done processing fold1
Done processing fold2
Done processing fold3


In [ ]:
# Display results for each fold
for i in range(k):
    print(f"\nFold {i+1}")
    print("Confusion Matrix:")
    print(conf_matrices_rf[i])
    print("\nClassification Report:")
    for label, metrics in classification_reports_rf[i].items():
        print(f"{label}: {metrics}")


Fold 1
Confusion Matrix:
[[274572  79250]
 [101606 252906]]

Classification Report:
0: {'precision': 0.7298991434905816, 'recall': 0.7760173194431098, 'f1-score': 0.7522520547945205, 'support': 353822.0}
1: {'precision': 0.7614072905502234, 'recall': 0.7133919303154759, 'f1-score': 0.7366179871495395, 'support': 354512.0}
accuracy: 0.7446741226596493
macro avg: {'precision': 0.7456532170204024, 'recall': 0.7447046248792928, 'f1-score': 0.7444350209720301, 'support': 708334.0}
weighted avg: {'precision': 0.7456685633269974, 'recall': 0.7446741226596493, 'f1-score': 0.7444274062686027, 'support': 708334.0}

Fold 2
Confusion Matrix:
[[273493  80958]
 [ 99231 254651]]

Classification Report:
0: {'precision': 0.7337681501593672, 'recall': 0.7715960739284132, 'f1-score': 0.7522068277924846, 'support': 354451.0}
1: {'precision': 0.7587728577004789, 'recall': 0.7195929716685223, 'f1-score': 0.7386637389030458, 'support': 353882.0}
accuracy: 0.7456154097013693
macro avg: {'precision': 0.746270

In [ ]:
# Calculate average accuracy
accuracies_rf = [report['accuracy'] for report in classification_reports_rf]
average_accuracy_rf = np.mean(accuracies_rf)
print(f"\nAverage Accuracy RF: {average_accuracy_rf}")

# Caluculate average standard deviation
std_dev_rf = np.std(accuracies_rf)
print(f"Standard Deviation: {std_dev_rf}")


Average Accuracy RF: 0.7454207062336864
Standard Deviation: 0.000547682427792883


In [ ]:
# Collect the recall for the '1' class from each fold
recall_class_1_rf = [report['1']['recall'] for report in classification_reports_rf]

# Calculate the average recall for the '1' class across all folds
average_recall_class_1_rf = np.mean(recall_class_1_rf)
print(f"\nAverage Recall for class 1: {average_recall_class_1_rf}")


Average Recall for class 1: 0.7165659120604816


### Support Vector Machine

In [12]:
from sklearn.model_selection import train_test_split
from thundersvm import SVC

FileNotFoundError: Please build the library first!

In [22]:
# Initialize SVC

svc = SVC(kernel='rbf', random_state=45)

# choosing rbf cause not linearly separable

In [ ]:
# Lists to store results
conf_matrices_svm = []
classification_reports_svm = []
count = 0

# Manually loop through each fold
for train_index, test_index in kf.split(X_trainval):
    count += 1

    print("Starting fold " + str(count))
    # Split the data for this fold
    X_train, X_val = X_trainval.iloc[train_index], X_trainval.iloc[test_index]
    y_train, y_val = y_trainval.iloc[train_index], y_trainval.iloc[test_index]

    # Train the model
    svc.fit(X_train, y_train)
    print("Model fitted")

    # Make predictions
    y_pred_svm = svc.predict(X_val)

    # Store confusion matrix and classification report for each fold
    conf_matrices_svm.append(confusion_matrix(y_val, y_pred_svm))
    classification_reports_svm.append(classification_report(y_val, y_pred_svm, output_dict=True))

    print("Done processing fold " + str(count))

Starting fold 1


In [ ]:
# Display results for each fold
for i in range(k):
    print(f"\nFold {i+1}")
    print("Confusion Matrix:")
    print(conf_matrices_svm[i])
    print("\nClassification Report:")
    for label, metrics in classification_reports_svm[i].items():
        print(f"{label}: {metrics}")


Fold 1
Confusion Matrix:
[[33869  7839]
 [19892 21734]]

Classification Report:
0: {'precision': 0.6299920016368743, 'recall': 0.81205044595761, 'f1-score': 0.7095287475515613, 'support': 41708.0}
1: {'precision': 0.7349271294762114, 'recall': 0.5221255945803104, 'f1-score': 0.6105141926150648, 'support': 41626.0}
accuracy: 0.6672306621547027
macro avg: {'precision': 0.6824595655565429, 'recall': 0.6670880202689602, 'f1-score': 0.660021470083313, 'support': 83334.0}
weighted avg: {'precision': 0.6824079378866673, 'recall': 0.6672306621547027, 'f1-score': 0.6600701848546235, 'support': 83334.0}

Fold 2
Confusion Matrix:
[[33671  8098]
 [19928 21636]]

Classification Report:
0: {'precision': 0.6282020186943786, 'recall': 0.8061241590653355, 'f1-score': 0.7061278416240249, 'support': 41769.0}
1: {'precision': 0.7276518463711577, 'recall': 0.5205466268886536, 'f1-score': 0.6069174450896239, 'support': 41564.0}
accuracy: 0.663686654746619
macro avg: {'precision': 0.6779269325327681, 'recal

In [ ]:
# Calculate average accuracy
accuracies_svm = [report['accuracy'] for report in classification_reports_svm]
average_accuracy_svm = np.mean(accuracies_svm)
print(f"\nAverage Accuracy SVM: {average_accuracy_svm}")

# Caluculate average standard deviation
std_dev_svm = np.std(accuracies_svm)
print(f"Standard Deviation: {std_dev_svm}")


Average Accuracy RF: 0.6657119939253271
Standard Deviation: 0.0014905331742503187


In [ ]:
# Collect the recall for the '1' class from each fold
recall_class_1_svm = [report['1']['recall'] for report in classification_reports_svm]

# Calculate the average recall for the '1' class across all folds
average_recall_class_1_svm = np.mean(recall_class_1_svm)
print(f"\nAverage Recall for class 1: {average_recall_class_1_svm}")

### MLP

In [ ]:
from tensorflow.keras import layers, optimizers, models, callbacks
from tensorflow.keras.optimizers.schedules import ExponentialDecay

In [ ]:
# Define parameters
initial_learning_rate = 5e-4
decay_rate = 0.1  # Decay rate per step (you can adjust this)

# Define the learning rate schedule
lr_schedule = ExponentialDecay(
    initial_learning_rate,
    decay_steps=10000,
    decay_rate=decay_rate)

# Define early stopping
early_stopping = callbacks.EarlyStopping(
    monitor='accuracy',                 # Watch validation loss
    patience=3,                         # Stop after 3 epochs with no improvement
    restore_best_weights=True,          # Restore weights from the best epoch
    min_delta=0.001                     # Minimum change in accuracy to qualify as an improvement
)

# Function to build the model
def build_model():
    model_mlp = models.Sequential()
    model_mlp.add(layers.Dense(64, activation='relu', input_shape=(X.shape[1],)))
    model_mlp.add(layers.BatchNormalization())
    model_mlp.add(layers.Dense(64, activation='relu'))
    model_mlp.add(layers.BatchNormalization())
    model_mlp.add(layers.Dense(1, activation='sigmoid'))
    model_mlp.compile(optimizer=optimizers.Adam(learning_rate=lr_schedule), loss='binary_crossentropy', metrics=['accuracy'])
    return model_mlp

In [ ]:
# Convert data to float32 (required by TensorFlow)
X_trainval = np.array(X_trainval, dtype=np.float32)
y_trainval = np.array(y_trainval, dtype=np.float32)

# Lists to store results
conf_matrices_mlp = []
classification_reports_mlp = []
count = 0

# Manually loop over each fold
for train_index, test_index in kf.split(X_trainval):
    count += 1
    print(f"\nFold {count}")

    # Split data for this fold
    X_train, X_val = X_trainval[train_index], X_trainval[test_index]
    y_train, y_val = y_trainval[train_index], y_trainval[test_index]

    # Build and train the model
    model_mlp = build_model()
    model_mlp.fit(X_train, y_train, epochs=20, batch_size=32, verbose=1, callbacks=[early_stopping])

    # Make predictions
    y_pred_mlp = model_mlp.predict(X_val)
    y_pred_mlp = (y_pred_mlp > 0.5).astype(int)  # Convert probabilities to binary class predictions

    # Store confusion matrix and classification report for each fold
    conf_matrices_mlp.append(confusion_matrix(y_val, y_pred_mlp))
    classification_reports_mlp.append(classification_report(y_val, y_pred_mlp, output_dict=True))


Fold 1
Epoch 1/20
44271/44271 ━━━━━━━━━━━━━━━━━━━━ 54s 1ms/step - accuracy: 0.6306 - loss: 0.6169
Epoch 2/20
44271/44271 ━━━━━━━━━━━━━━━━━━━━ 50s 1ms/step - accuracy: 0.6441 - loss: 0.6020
Epoch 3/20
44271/44271 ━━━━━━━━━━━━━━━━━━━━ 52s 1ms/step - accuracy: 0.6429 - loss: 0.6023
Epoch 4/20
44271/44271 ━━━━━━━━━━━━━━━━━━━━ 55s 1ms/step - accuracy: 0.6438 - loss: 0.6022
Epoch 5/20
44271/44271 ━━━━━━━━━━━━━━━━━━━━ 53s 1ms/step - accuracy: 0.6440 - loss: 0.6018
22136/22136 ━━━━━━━━━━━━━━━━━━━━ 18s 823us/step

Fold 2
Epoch 1/20


c:\Anaconda\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


44271/44271 ━━━━━━━━━━━━━━━━━━━━ 53s 1ms/step - accuracy: 0.6293 - loss: 0.6174
Epoch 2/20
44271/44271 ━━━━━━━━━━━━━━━━━━━━ 52s 1ms/step - accuracy: 0.6432 - loss: 0.6023
Epoch 3/20
44271/44271 ━━━━━━━━━━━━━━━━━━━━ 51s 1ms/step - accuracy: 0.6429 - loss: 0.6027
22136/22136 ━━━━━━━━━━━━━━━━━━━━ 19s 844us/step

Fold 3
Epoch 1/20


c:\Anaconda\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


44271/44271 ━━━━━━━━━━━━━━━━━━━━ 55s 1ms/step - accuracy: 0.6265 - loss: 0.6183
Epoch 2/20
44271/44271 ━━━━━━━━━━━━━━━━━━━━ 56s 1ms/step - accuracy: 0.6411 - loss: 0.6033
Epoch 3/20
44271/44271 ━━━━━━━━━━━━━━━━━━━━ 53s 1ms/step - accuracy: 0.6409 - loss: 0.6038
22136/22136 ━━━━━━━━━━━━━━━━━━━━ 20s 893us/step


In [ ]:
# Calculate average accuracy
accuracies_mlp = [report['accuracy'] for report in classification_reports_mlp]
average_accuracy_mlp = np.mean(accuracies_mlp)
print(f"\nAverage Accuracy: {average_accuracy_mlp}")

# Caluculate average standard deviation
std_dev_mlp= np.std(accuracies_mlp)
print(f"Standard Deviation: {std_dev_mlp}")


Average Accuracy: 0.6544847063445941
Standard Deviation: 0.0007639610847939577


In [ ]:
import numpy as np

# Collect the recall for the '1.0' class from each fold
recall_class_1_mlp = [report_mlp['1.0']['recall'] for report_mlp in classification_reports_mlp]

# Calculate the average recall for the '1.0' class across all folds
average_recall_class_1_mlp = np.mean(recall_class_1_mlp)
print(f"\nAverage Recall for class 1.0: {average_recall_class_1_mlp}")



Average Recall for class 1.0: 0.5632927499334602


## Dataframe with results

In [ ]:

# List of model names
model_names = ['Logistic Regression', 'Random Forest', 'MLP']

# Collect the accuracy and recall values by calling the variables
accuracies = [average_accuracy_lr, average_accuracy_rf, average_accuracy_mlp]
recalls = [average_recall_class_1_lr, average_recall_class_1_rf, average_recall_class_1_mlp]

# Create a dictionary for the DataFrame
data = {
    'Model': model_names,
    'Accuracy': accuracies,
    'Recall': recalls
}

# Create the DataFrame
df_results = pd.DataFrame(data)

In [ ]:
df_results

,Model,Accuracy,Recall
0,Logistic Regression,0.598670,0.448558
1,Random Forest,0.745421,0.716566
2,MLP,0.654485,0.563293


## Random search on best model

After looking at the df of above, it can be seen that the best model is the Random Forest model. I therefore will perform random search on this model

In [ ]:
# Further splitting trainval into train and validation set to tune hyperparameters on the validation set
# Training set: 70%, Validation set: 15%, Test set: 15% (still the same test set)

X_train, X_val, y_train, y_val = train_test_split(X_trainval, y_trainval, test_size=(15/85), random_state=42)

In [ ]:
print(X_train.shape)
print(f"The percentage of the original dataset that is used for training is: {X_train.shape[0]/df50.shape[0]*100} %")

(1750000, 45)
The percentage of the original dataset that is used for training is: 70.0 %


In [ ]:
from scipy.stats import randint  # For defining distributions for random search

param_dist = {
    'n_estimators': randint(50, 200),  # Number of trees in the forest (uniform distribution between 50 and 200)
    'max_depth': [None] + list(randint(1, 30).rvs(10)),  # Random depth values including None
    'min_samples_split': randint(2, 10),  # Minimum samples required to split an internal node
    'min_samples_leaf': randint(1, 4)  # Minimum samples required to be at a leaf node
}

In [ ]:
# Initialize the Random Forest Classifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV

rf_model_search = RandomForestClassifier(random_state=42)  # Random state for reproducibility

In [ ]:
# Set up RandomizedSearchCV
random_search = RandomizedSearchCV(estimator=rf_model_search,
                           param_distributions=param_dist,
                           n_iter=20,          # Number of parameter settings that are sampled
                           scoring='recall',   # Use recall as the evaluation metric
                           cv=k,
                           verbose=2,
                           n_jobs=-1)          # Use all available cores

# Fit RandomizedSearchCV
random_search.fit(X_train, y_train)

# Get the best parameters and best score
print("Best Parameters:", random_search.best_params_)
print("Best Cross-Validation Score:", random_search.best_score_)

Fitting 3 folds for each of 20 candidates, totalling 60 fits
Best Parameters: {'max_depth': 27, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 53}
Best Cross-Validation Score: 0.6859427727743856


In [ ]:
y_pred_rf_random = random_search.predict(X_val)

print("Accuracy on Validation Set:", accuracy_score(y_val, y_pred_rf_random))
print(confusion_matrix(y_val, y_pred_rf_random))
print(classification_report(y_val, y_pred_rf_random))

Accuracy on Validation Set: 0.7395093333333334
[[147891  39282]
 [ 58402 129425]]
              precision    recall  f1-score   support

           0       0.72      0.79      0.75    187173
           1       0.77      0.69      0.73    187827

    accuracy                           0.74    375000
   macro avg       0.74      0.74      0.74    375000
weighted avg       0.74      0.74      0.74    375000

